In [3]:
import importlib
import cv2
import json
import os
import sys

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../')
sys.path.append(NOMEROFF_NET_DIR)

def compare(model_path="../../../models/anpr_ocr_eu_2-gpu.h5", 
           text_detector_name = "eu",
           img_format = "png",
           root_dir='/mnt/data/home/nn/datasets/autoriaNumberplateOcrEu-2019-02-19/train', # ../../../datasets/ocr/kz/kz2
           predicted_part_size=1000,
           replace_tamplate = {'moderation': {'isModerated': 1, 'moderatedBy': 'ApelSYN'}}):
    text_detector_module = importlib.import_module("NomeroffNet.TextDetectors."+text_detector_name)
    text_detector = getattr(text_detector_module, text_detector_name)()
    text_detector.load(model_path)

    ann_dir = os.path.join(root_dir, "ann")
    jsons = []
    jsons_paths = []
    for dirName, subdirList, fileList in os.walk(ann_dir):
        for fname in fileList:
            fname = os.path.join(ann_dir, fname)
            jsons_paths.append(fname)
            with open(fname) as jsonF:
                jsonData = json.load(jsonF)
            jsons.append(jsonData)
    print("LOADED {} ANNOTATIONS".format(len(jsons)))

    img_dir = os.path.join(root_dir, "img")
    imgs = []                
    for j in jsons:
        img_path =os.path.join(img_dir, "{}.{}".format(j["name"], img_format))
        img = cv2.imread(img_path)
        imgs.append(img)
    print("LOADED {} IMAGES".format(len(imgs)))

    predicted = []
    N = int(len(imgs) / predicted_part_size) + 1
    for i in range(N):
        part = i*predicted_part_size
        part_imgs = imgs[part:part+predicted_part_size]
        predicted_part = text_detector.predict(part_imgs)
        predicted += predicted_part

    print("PREDICTED {} IMAGES".format(len(predicted)))

    err_cnt = 0
    for i in range(len(jsons_paths)):
        json_path      = jsons_paths[i]
        predicted_item = predicted[i]
        jsonData       = jsons[i]
        jsonData["moderation"]["predicted"] = predicted_item
        if jsonData["description"] == jsonData["moderation"]["predicted"]:
            #jsonData.update(replace_tamplate)
            jsonData["moderation"]["isModerated"] = 1
        else:
            print("Predicted '{}', real: '{}' in file {}".format(jsonData["moderation"]["predicted"],jsonData["description"], json_path))
            err_cnt = err_cnt+1

        with open(json_path, "w") as jsonWF:
            json.dump(jsonData, jsonWF)

    print("Error detection count: {}".format(err_cnt))
    print("Accuracy: {}".format(1-err_cnt/len(predicted)))

In [4]:
compare()

LOADED 6255 ANNOTATIONS
LOADED 6255 IMAGES
PREDICTED 6255 IMAGES
Predicted 'EZ7766', real: 'EZ677LX' in file /mnt/data/home/nn/datasets/autoriaNumberplateOcrEu-2019-02-19/train/ann/260250990.json
Predicted '81129IB', real: '81929IB' in file /mnt/data/home/nn/datasets/autoriaNumberplateOcrEu-2019-02-19/train/ann/259310102orig.jpegzone1.json
Predicted '32KKD34', real: '2KKD34' in file /mnt/data/home/nn/datasets/autoriaNumberplateOcrEu-2019-02-19/train/ann/260231086.json
Predicted '7575OXM', real: '75750XM' in file /mnt/data/home/nn/datasets/autoriaNumberplateOcrEu-2019-02-19/train/ann/259980445.json
Predicted '8B3782', real: 'RB3782' in file /mnt/data/home/nn/datasets/autoriaNumberplateOcrEu-2019-02-19/train/ann/260065301.json
Predicted 'LCH51557', real: 'LCH52557' in file /mnt/data/home/nn/datasets/autoriaNumberplateOcrEu-2019-02-19/train/ann/259977823.json
Predicted '75860O', real: '75860OA' in file /mnt/data/home/nn/datasets/autoriaNumberplateOcrEu-2019-02-19/train/ann/261392950.json
